# Example to use the custom Container for mono reconstruction

In [45]:
import os

from lstchain.io.containers import DL1ParametersContainer
from lstchain.reco.utils import guess_type
from lstchain.gammalearn import load_model, load_camera_parameters

from ctapipe.utils import get_dataset_path
from ctapipe.io import HDF5TableWriter, HDF5TableReader
from ctapipe.calib import CameraCalibrator
from ctapipe.image import tailcuts_clean
from ctapipe.io import event_source
from ctapipe.image import hillas_parameters

from astropy import units

import torch

## Options

In [2]:
infile = get_dataset_path('gamma_test_large.simtel.gz')   

dl1_parameters_filename = 'dl1.h5'

allowed_tels = {1} # select LST1 only
max_events = 300 # limit the number of events to analyse in files - None if no limit

cal = CameraCalibrator(r1_product='HESSIOR1Calibrator', extractor_product='NeighbourPeakIntegrator')

cleaning_method = tailcuts_clean
cleaning_parameters = {'boundary_thresh': 0,
                       'picture_thresh': 6,
                       'min_number_picture_neighbors': 2}

## Load the trained model

In [47]:
if not os.path.isfile('lst_bootcamp_gl_data.tar.gz'):
    !wget https://gitlab.lapp.in2p3.fr/GammaLearn/GammaLearn/raw/master/share/lst_bootcamp_gl_data.tar.gz
    !tar xvzf lst_bootcamp_gl_data.tar.gz

In [ ]:
exp_name = 'R_0157'
exps_path = 'lst_bootcamp_gl_data/'
camera_parameters_path = 'camera_parameters.h5'

network = load_model(exps_path, exp_name, 100, camera_parameters_path)

## R0 to DL1

In [3]:
dl1_container = DL1ParametersContainer() 

In [28]:
with HDF5TableWriter(filename=dl1_parameters_filename, group_name='events', overwrite=True) as writer:

    source = event_source(infile)
    source.allowed_tels = allowed_tels
    source.max_events = max_events

    for i, event in enumerate(source):
        if i%100==0:
            print(i)
        cal.calibrate(event)

        for telescope_id, dl1 in event.dl1.tel.items():
            tel = event.inst.subarray.tels[telescope_id]
            camera = tel.camera

            image = dl1.image[0]
            
            peakpos = dl1.peakpos[0]

            data = torch.tensor([image, peakpos], dtype=torch.float).unsqueeze(0)
            prediction = network(data).squeeze(0).detach().numpy()

            ## Fill container ##
            dl1_container.fill_mc(event)
            dl1_container.fill_event_info(event)
            dl1_container.set_mc_core_distance(event, telescope_id)
            dl1_container.mc_type = guess_type(infile)
            dl1_container.set_source_camera_position(event, telescope_id)
            
            event.dl2.energy['gl'].energy = 10**prediction[0] * units.TeV
            event.dl2.shower['gl'].core_x = prediction[1] * units.km
            event.dl2.shower['gl'].core_y = prediction[2] * units.km
            event.dl2.shower['gl'].alt = prediction[3] * units.rad
            event.dl2.shower['gl'].az = prediction[4] * units.rad
            

            ## Save parameters for later training ##
            writer.write(camera.cam_id, [dl1_container, event.dl2.energy['gl'], event.dl2.shower['gl']])
                
                

0
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled
mc information not filled


In [29]:
# The file has been created
!ls -lsh dl1.h5

76K -rw-rw-r-- 1 mikael mikael 75K nov.  21 11:21 dl1.h5


## Loading hdf5 file is very easy with pandas

In [31]:
import pandas as pd
df = pd.read_hdf(dl1_parameters_filename, key='events/LSTCam')

In [33]:
df

,alt,alt_uncert,average_size,az,az_uncert,core_uncert,core_x,core_y,energy,energy_uncert,...,mc_alt,mc_az,mc_core_distance,mc_core_x,mc_core_y,mc_energy,mc_h_first_int,obs_id,src_x,src_y
0,69.284524,0.0,0.0,185.018172,0.0,0.0,-40.240906,47.046527,0.028682,-1.0,...,1.203404,0.046534,174.622923,-51.531204,-106.752579,0.010250,19824.572266,7514,-0.503035,0.467968
1,68.108734,0.0,0.0,181.784720,0.0,0.0,-56.052573,14.206529,0.105698,-1.0,...,1.174510,6.277897,239.585294,-211.719864,-78.682312,0.120758,24300.533203,7514,-1.323027,-0.057215
2,71.825702,0.0,0.0,179.919737,0.0,0.0,-124.887533,21.860175,0.042027,-1.0,...,1.252226,6.271463,31.903575,10.729630,56.425159,0.031681,17258.339844,7514,0.854726,-0.102846
3,69.998681,0.0,0.0,179.171257,0.0,0.0,-1.305323,-18.483631,0.033076,-1.0,...,1.225230,6.199299,262.288108,24.686068,-193.453491,0.126791,41954.234375,7514,0.129366,-0.795008
4,70.938249,0.0,0.0,175.669337,0.0,0.0,-184.870660,-42.121582,0.100841,-1.0,...,1.236999,6.234098,276.822666,173.315231,263.141388,0.146845,35380.851562,7514,0.437997,-0.450242
5,68.867288,0.0,0.0,175.040973,0.0,0.0,-29.152911,165.199965,0.052400,-1.0,...,1.199205,6.179521,99.050748,40.452858,-13.463385,0.065823,22263.962891,7514,-0.579898,-1.053016
6,69.401224,0.0,0.0,179.197021,0.0,0.0,30.861361,71.305625,0.033439,-1.0,...,1.219127,6.193979,230.428093,17.040657,-162.431519,0.016580,13893.394531,7514,-0.036868,-0.859664
7,70.003196,0.0,0.0,177.143620,0.0,0.0,40.508077,23.238026,0.022378,-1.0,...,1.234354,6.245500,110.780623,73.603035,124.252159,0.009404,15173.717773,7514,0.359682,-0.348327
8,71.830470,0.0,0.0,179.807817,0.0,0.0,-52.935332,263.431162,0.119079,-1.0,...,1.255599,6.279010,251.544837,52.627186,-175.832092,0.248144,28720.640625,7514,0.948764,-0.036260
9,69.472047,0.0,0.0,170.639613,0.0,0.0,56.875497,363.173544,0.355095,-1.0,...,1.207581,6.135493,275.341046,-117.548683,-192.481934,0.407132,36972.437500,7514,-0.294832,-1.465980
